# ASSIGNMENT 3

In this kernel, I try to make predictions where the prediction task is to determine whether a person makes over 50K a year. I implement Random Forest Classification with Python and Scikit-Learn. So, to answer the question, I build a Random Forest classifier to predict whether a person makes over 50K a year.

I have used the Income classification data set for this project.

## Import libraries

In [ ]:
#dataframe
import pandas as pd
import numpy as np

#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Binarizer

#Classification Algorithms
import lightgbm as lgb
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# suppressing warnings
import warnings
warnings.filterwarnings('ignore')

## Import dataset

In [ ]:
X_test = pd.read_csv('test.csv')
X_test

In [ ]:
X_train = pd.read_csv('train.csv')
X_train

### Exploratory data analysis

* Now, I will explore the data to gain insights about the data

In [ ]:
# print the shape
print('The shape of the dataset : ', X_train.shape)

We can see that there are 43957 instances and 15 attributes in the data set.

In [ ]:
#Listing Dataframe columns
X_train.columns

* Rename column names

We can see that the dataset does not have proper column names. The column names contain underscore. We should give proper names to the columns. I will do it as follows:

* View summary of dataset

In [ ]:
#Determining dataframe columns characteristics
X_train.info()

In [ ]:
X_train.head(5)

Findings
We can see that the dataset contains 9 character variables and 6 numerical variables.

income is the target variable.

There are no missing values in the dataset. I will explore this later.

* View statistical properties of dataset

In [ ]:
#Descriptive Analytics of quantitative variables in the dataframe
X_train.describe().T

We can see that the above df.describe().T command presents statistical properties in horizontal form.

In [ ]:
#calculating mode for categorical variables:
def mode(list_of_nums):
    average = list_of_nums.mode()
    return average

categorical_var = list(X_train.select_dtypes(include=object).columns)

#printing mode of categorical variables
for item in categorical_var:
    print(item + " mode: "+ mode(X_train[item]))

In [ ]:
#Checking if any cells has missing values
X_train.isnull().sum()

Interpretation
We can see that there are 3 columns missing values in the dataset.

In [ ]:
## Dealing with NaN value
missing_val_col = ['workclass','occupation','native-country']

for i in missing_val_col:
    X_train[i] = X_train[i].fillna(method='ffill')
X_train.isnull().sum()

### Remove white space

In [ ]:
X_train.columns = X_train.columns.str.lstrip()
X_test.columns = X_test.columns.str.lstrip()

## Explore Categorical Variables

### Find categorical variables

In [ ]:
categorical = [var for var in X_train.columns if X_train[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

In [ ]:
X_train[categorical].head()

* Frequency distribution of categorical variables

Now, we will check the frequency distribution of categorical variables.

In [ ]:
for var in categorical: 
    
    print(X_train[var].value_counts())

* Percentage of frequency distribution of values

In [ ]:
for var in categorical:
    
     print(X_train[var].value_counts()/np.float(len(X_train)))

In [ ]:
# view the unique values

X_train['income_>50K'].unique()

In [ ]:
# Checking the distribution of target variable
display(X_train['income'].value_counts(), X_train['income'].value_counts(normalize = True)*100)

Visualize income variable

In [ ]:
# visualize frequency distribution of income variable

f,ax=plt.subplots(1,2,figsize=(18,8))

ax[0] = X_train['income'].value_counts().plot.pie(explode=[0,0],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Income Share')


#f, ax = plt.subplots(figsize=(6, 8))
ax[1] = sns.countplot(x="income", data=X_train, palette="Set1")
ax[1].set_title("Frequency distribution of income variable")

plt.show()

Visualize income with gender variable

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
ax = sns.countplot(x="income", hue="gender", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of income variable with gender")
plt.show()

Interpretation:
We can see that males make more money than females in both the income categories.

Visualize income with race

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
ax = sns.countplot(x="income", hue="race", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of income variable with race")
plt.show()

Interpretation¶
* We can see that whites make more money than non-whites in both the income categories.

Explore workclass variable

In [ ]:
# view the unique labels

X_train.workclass.unique()

In [ ]:
# view frequency distribution of values

X_train.workclass.value_counts()

Visualize workclass variable

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
ax = X_train.workclass.value_counts().plot(kind="bar", color="green")
ax.set_title("Frequency distribution of workclass variable")
ax.set_xticklabels(X_train.workclass.value_counts().index, rotation=30)
plt.show()

Interpretation¶
* We can see that there are lot more private workers than other category of workers.

Visualize workclass variable with income variable

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))
ax = sns.countplot(x="workclass", hue="income", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of workclass variable wrt income")
ax.legend(loc='upper right')
plt.show()

Interpretation
* We can see that workers make less than equal to 50k in most of the working categories.

* But this trend is more appealing in Private workclass category.

Visualize workclass variable with education variable

In [ ]:
f, ax = plt.subplots(figsize=(12, 8))
ax = sns.countplot(x="workclass", hue="education", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of workclass variable with education")
ax.legend(loc='upper right')
plt.show()

Explore occupation variable

In [ ]:
# view unique labels
X_train.occupation.unique()

In [ ]:
#check the frequency distribution of values

X_train.occupation.value_counts()

In [ ]:
# visualize frequency distribution of `occupation` variable

f, ax = plt.subplots(figsize=(12, 8))
ax = sns.countplot(x="occupation", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of occupation variable")
ax.set_xticklabels(X_train.occupation.value_counts().index, rotation=30)
plt.show()

Explore native_country variable

In [ ]:
# view unique labels 

X_train.native_country.unique()

In [ ]:
# check frequency distribution of values

X_train.native_country.value_counts()

In [ ]:
# visualize frequency distribution of `native_country` variable

f, ax = plt.subplots(figsize=(16, 12))
ax = sns.countplot(x="native_country", data=X_train, palette="Set1")
ax.set_title("Frequency distribution of native_country variable")
ax.set_xticklabels(X_train.native_country.value_counts().index, rotation=90)
plt.show()

In [ ]:
# check for cardinality in categorical variables

for var in categorical:
    
    print(var, ' contains ', len(X_train[var].unique()), ' labels')

## Explore Numerical Variables

* Find numerical variables

In [ ]:
numerical = [var for var in X_train.columns if X_train[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :\n\n', numerical)

In [ ]:
X_train[numerical].head()

Summary of numerical variables¶
* There are 6 numerical variables.

* These are given by age, fnlwgt, education_num,capital_gain, capital_loss and hours_per_week.

* All of the numerical variables are of discrete data type.

Explore numerical variables

Explore age variable

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
x = X_train['age']
ax = sns.distplot(x, bins=10, color='blue')
ax.set_title("Distribution of age variable")
plt.show()

Detect outliers in age variable with boxplot

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
x = X_train['age']
ax = sns.boxplot(x)
ax.set_title("Visualize outliers in age variable")
plt.show()

Explore relationship between age and income variables

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
ax = sns.boxplot(x="income", y="age", data=X_train)
ax.set_title("Visualize income wrt age variable")
plt.show()

Interpretation¶
* As expected, younger people make less money as compared to senior people.

Visualize income with age and sex variable

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
ax = sns.boxplot(x="income", y="age", hue="gender", data=X_train)
ax.set_title("Visualize income with age and sex variable")
ax.legend(loc='upper right')
plt.show()

Interpretation
* Senior people make more money than younger people.

Visualize relationship between race and age

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x ='race', y="age", data = X_train)
plt.title("Visualize age wrt race")
plt.show()

Interpretation¶
* Whites are more older than other groups of people.

Find out the correlations

In [ ]:
# plot correlation heatmap to find out correlations

X_train.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

Interpretation
* We can see that there is no strong correlation between variables.

In [ ]:
sns.pairplot(X_train, hue="income")
plt.show()

In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

## Feature Engineering

#### Display categorical variables in training set

In [ ]:
ord_features = []

#categorical features
cat_features = list(X_train.select_dtypes(include=['object']).drop(ord_features, axis=1))

#numeric features
num_features = list(X_train.select_dtypes(include=['number']))

In [ ]:
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in ord_features:
    X_train[col] = label_encoder.fit_transform(X_train[col])

In [ ]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_X_train = pd.DataFrame(OH_encoder.fit_transform(X_train[cat_features]))
OH_X_test = pd.DataFrame(OH_encoder.fit_transform(X_test[cat_features]))

# One-hot encoding removed index; put it back
OH_X_train.index = X_train.index
OH_X_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_col_X_train = X_train.drop(cat_features, axis=1)
num_col_X_test = X_test.drop(cat_features, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train_finish = pd.concat([num_col_X_train, OH_X_train], axis=1)
OH_X_test_finish = pd.concat([num_col_X_test, OH_X_test], axis=1)

OH_X_train_finish

#### Display numerical variables in training set

#### Encode categorical variables

### Declare feature vector and target variable

In [ ]:
X=OH_X_train_finish.drop(['income_>50K'],axis=1)
y=OH_X_train_finish[['income_>50K']]

from collections import Counter
print(sorted(Counter(y).items()))

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=21000, n_features=13, n_informative=2,
                            n_redundant=0, n_repeated=0, n_classes=2,
                            n_clusters_per_class=1,
#                             weights=[0.01, 0.05, 0.94],
                            class_sep=0.8, random_state=0)

In [ ]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_resampled, Y_resampled = cc.fit_resample(X, y)
print(sorted(Counter(Y_resampled).items()))

### Split data into separate training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# X_test[categorical_cols] = X_test[categorical_cols].apply(lambda col: le.fit_transform(col))
# X_test[categorical_cols].head(5)

In [ ]:
X_train.shape
X_test.shape

## Modelling

In [ ]:
def train_and_plot(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)

    print('Model accuracy for train set: {0:.3f}'.format(model.score(X_train, y_train)))
    print('Model accuracy for test set: {0:.3f}'.format(model.score(X_test, y_test)))

    y_pred = model.predict(X_test)

    # Classification Report
    print('\n{}'.format(classification_report(y_test, y_pred)))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print('\nConfusion Matrix:\n', cm)

    # Accuracy Score
    auc = accuracy_score(y_test, y_pred)
    print('\nAccuracy Score: ', auc.round(3))

    # ROC Curve
    model_roc_auc = roc_auc_score(y_test, model.predict(X_test))
    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    plt.figure(figsize = (12.8 , 6))
    plt.plot(fpr, tpr, label = 'AUC = {})'.\
             format(model_roc_auc.round(4)))
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc = 'lower right')
    plt.show()
    
    return model

In [ ]:
#Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb

#Converting the dataframe into XGBoost’s Dmatrix object
dtrain = xgb.DMatrix(X,y)

#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,120)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=5, init_points=8, acq='ei')

In [ ]:
#Extracting the best parameters
params = xgb_bo.max['params']
print(params)

In [ ]:
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']= int(params['n_estimators'])

In [ ]:
# params['gamma']= float(params['gamma'])
# params['learning_rate']= float(params['learning_rate'])

In [ ]:
# XGBOOST
import xgboost
xgb1 = xgboost.XGBClassifier(gamma=0.8,
                           learning_rate=0.2,
                           max_depth=8,
                           n_estimators=119)
model9 = train_and_plot(xgb1, X_train, y_train, X_test, y_test)

In [ ]:
# XGBOOST
import xgboost
XGB = xgboost.XGBClassifier()
model1 = train_and_plot(XGB, X_train, y_train, X_test, y_test)

In [ ]:
# Logistic Regression
lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr') # creates a lR instance
model2 = train_and_plot(lr, X_train, y_train, X_test, y_test)

In [ ]:
# hyperparameter tuning
LR = LogisticRegression()

params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'max_iter': [100, 110, 120, 130, 140]}

skf = StratifiedKFold(n_splits = 10)
LR_hyper = GridSearchCV(LR, param_grid = params, n_jobs = -1, cv = skf)

model3 = train_and_plot(LR_hyper, X_train, y_train, X_test, y_test)

In [ ]:
# Decision Trees
dt = DecisionTreeClassifier(criterion = 'gini', splitter='best', max_depth=8)
model4 = train_and_plot(dt, X_train, y_train, X_test, y_test)

In [ ]:
# Random Forests 
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=1000)
model5 = train_and_plot(rf, X_train, y_train, X_test, y_test)

In [ ]:
# K-NN
knn = KNeighborsClassifier(n_neighbors = 25)
model6 = train_and_plot(knn, X_train, y_train, X_test, y_test)

In [ ]:
# Import the model we are using
from sklearn.ensemble import GradientBoostingClassifier
# Instantiate model with 1000 decision trees
gbm = GradientBoostingClassifier(n_estimators = 119, 
                                 learning_rate = 0.05, 
                                 random_state = 1000, 
                                 n_iter_no_change = 5, 
                                 max_depth=8, 
                                 min_samples_split=3, 
                                 min_samples_leaf=3,
                                 verbose=1)
# Train the model on training data
model8 = train_and_plot(gbm, X_train, y_train, X_test, y_test)

In [ ]:
# computes the confusion matrix and performence indicators
def get_performances(actual_Y, pred_Y):
    cm = confusion_matrix(actual_Y, pred_Y.round())
    total = sum(sum(cm))
    accuracy = (cm[0,0]+cm[1,1])/total
    return accuracy

In [ ]:
pred_Y_xgbh = xgb1.predict(X_test) # predicts the Y given the fitted model XGBoost

pred_Y_xgb = XGB.predict(X_test) # predicts the Y given the fitted model XGBoost

pred_Y_lr = lr.predict(X_test) # predicts the Y given the fitted model lr

pred_Y_lrh = LR_hyper.predict(X_test) # predicts the Y given the fitted model lr with hypertunning

pred_Y_dt = dt.predict(X_test) # predicts the Y given the fitted model dt

pred_Y_rf = rf.predict(X_test) # predicts the Y given the fitted model rf

pred_Y_knn = knn.predict(X_test) # predicts the Y given the fitted model knn

pred_Y_gbm = gbm.predict(X_test) # predicts the Y given the fitted model Gradient Boosting

In [ ]:
# accuracy, sensitivity, and specificity for model XGboost
accuracy_xgbh = get_performances(y_test, pred_Y_xgbh)

# accuracy, sensitivity, and specificity for model XGboost
accuracy_xgb = get_performances(y_test, pred_Y_xgb)

# accuracy, sensitivity, and specificity for model lr
accuracy_lr = get_performances(y_test, pred_Y_lr)

# accuracy, sensitivity, and specificity for model lr
accuracy_lrh = get_performances(y_test, pred_Y_lrh)

# accuracy, sensitivity, and specificity for model dt
accuracy_dt = get_performances(y_test, pred_Y_dt)

# accuracy, sensitivity, and specificity for model rf
accuracy_rf = get_performances(y_test, pred_Y_rf)

# accuracy, sensitivity, and specificity for model knn
accuracy_knn = get_performances(y_test, pred_Y_knn)

# accuracy, sensitivity, and specificity for model Neural Networks
accuracy_gbm = get_performances(y_test, pred_Y_gbm)

In [ ]:
# Builds a dataframe using the performance indicators so that we can compare the models easily

perf = pd.DataFrame([accuracy_xgbh,accuracy_xgb,accuracy_lr, accuracy_lrh, accuracy_dt,accuracy_rf,accuracy_knn,accuracy_gbm], 
                    columns = ['accuracy'], 
                    index = ['XGBoost-Hyperparam','XGBoost','Logistic Regression','Logistic Regression Hyper',
                             'Decision Trees', 'Random Forest','K-NN','Gradient Boosting'])
perf

In [ ]:
# create the classifier with n_estimators = 100

clf = RandomForestClassifier(n_estimators=100, random_state=0)

# fit the model to the training set

clf.fit(X_train, y_train)

## Prediction

In [ ]:
y_pred = xgb1.predict(X_test)
print("Predicted values:")
print(y_pred)